In [ ]:
import sqlite3
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, LiteLLMModel
from dotenv import load_dotenv
import os
import json 

load_dotenv()
cohere_token = os.getenv("cohere_key")

# Define the agent
model = LiteLLMModel(
    model_id="command-r",  
    api_key=cohere_token,          
)
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)

# Prepare the database and fetch data
def choose_best_algorithm(db_file="train_mTSP.sqlite3"):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    cursor.execute("SELECT instance_id, nr_cities, nr_salesmen FROM instances")
    instances = cursor.fetchall()
    instances_with_columns = [
        {"instance_id": row[0], "nr_cities": row[1], "nr_salesmen": row[2]} for row in instances
    ]

    cursor.execute("SELECT instance_id, strategy, total_cost, time_taken, distance_gap FROM algorithms")
    algorithms_data = cursor.fetchall()
    algorithms_with_columns = [
        {
            "instance_id": row[0],
            "strategy": row[1],
            "total_cost": row[2],
            "time_taken": row[3],
            "distance_gap": row[4],
        }
        for row in algorithms_data
    ]

    # Prepare data for the agent
    data = {
        "instances": instances_with_columns,
        "algorithms": algorithms_with_columns,
    }
    data_str = json.dumps(data, indent=4)

    # Task for the agent
    task = f"""
    You are an AI agent tasked with selecting the best algorithm for solving each instance of the mTSP problem.
    Below is the data from the algorithms table, including column names for clarity:

    {data_str}

    You need to choose the best algorithm that should minimize total_cost, time_taken, and distance_gap from the algorithms table. 
    At the moment tha data contains the following strategies:
    1. "Greedy"
    2. "Bracnh and Cut"
    For each instance, print the selected algorithm from these and explain why it was chosen.
    """

    # Run the agent
    response = agent.run(task)
    print(response)
    
    conn.close()    
choose_best_algorithm()

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an AI agent tasked with selecting the best algorithm for solving each instance of the mTSP problem.     │
│     Below is the data from the algorithms table, including column names for clarity:                            │
│                                                                                                                 │
│     {                                                                                                           │
│     "instances": [                                                                                              │
│         {                                                                                                       │
│             "instance_id": 1,                                                                                   │
│             "nr_cities": 5,                                                                                     │
│             "nr_salesmen": 3                                                                                    │
│         },                                                                                                      │
│         {                                                                                                       │
│             "instance_id": 2,                                                                                   │
│             "nr_cities": 5,                                                                                     │
│             "nr_salesmen": 3                                                                                    │
│         },                                                                                                      │
│         {                                                                                                       │
│             "instance_id": 3,                                                                                   │
│             "nr_cities": 5,                                                                                     │
│             "nr_salesmen": 3                                                                                    │
│         },                                                                                                      │
│         {                                                                                                       │
│             "instance_id": 4,                                                                                   │
│             "nr_cities": 5,                                                                                     │
│             "nr_salesmen": 3                                                                                    │
│         },                                                                                                      │
│         {                                                                                                       │
│             "instance_id": 5,                                                                                   │
│             "nr_cities": 5,                                                                                     │
│             "nr_salesmen": 3                                                                                    │
│         },                                                                                                      │
│         {                                                                                                       │
│             "instance_id": 6,                                                                                   │
│             "nr_cities": 5,                           

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            To determine the best algorithm for each instance, we need to compare the performance of the "Greedy" 
and "Branch and Cut" strategies based on the metrics: **total_cost**, **time_taken**, and **distance_gap**. The 
goal is to minimize all three metrics. Below is the analysis for each instance:

---

### **Instance 1**
- **Greedy**:  
  `total_cost = 342.73`, `time_taken = 0.0`, `distance_gap = 71.91`  
- **Branch and Cut**:  
  `total_cost = 290.61`, `time_taken = 0.014`, `distance_gap = 122.24`  
- **Selected Algorithm**: **Branch and Cut**  
  **Reason**: Despite having a higher `distance_gap` and slightly higher `time_taken`, "Branch and Cut" achieves a 
significantly lower `total_cost` (290.61 vs 342.73), which is the most critical metric for optimization.

---

### **Instance 2**
- **Greedy**:  
  `total_cost = 342.73`, `time_taken = 0.0`, `distance_gap = 93.16`  
- **Branch and Cut**:  
  `total_cost = 299.95`, `time_taken = 0.024`, `distance_gap = 95.33`  
- **Selected Algorithm**: **Branch and Cut**  
  **Reason**: "Branch and Cut" has a lower `total_cost` (299.95 vs 342.73), despite slightly higher `distance_gap` 
and `time_taken`.

---

### **Instance 3**
- **Greedy**:  
  `total_cost = 468.69`, `time_taken = 0.0`, `distance_gap = 114.77`  
- **Branch and Cut**:  
  `total_cost = 439.57`, `time_taken = 0.017`, `distance_gap = 118.11`  
- **Selected Algorithm**: **Branch and Cut**  
  **Reason**: "Branch and Cut" achieves a lower `total_cost` (439.57 vs 468.69), which outweighs the slightly 
higher `distance_gap` and `time_taken`.

---

### **Instance 4**
- **Greedy**:  
  `total_cost = 260.85`, `time_taken = 0.0`, `distance_gap = 125.97`  
- **Branch and Cut**:  
  `total_cost = 255.66`, `time_taken = 0.016`, `distance_gap = 124.83`  
- **Selected Algorithm**: **Branch and Cut**  
  **Reason**: "Branch and Cut" has a lower `total_cost` (255.66 vs 260.85) and a slightly better `distance_gap`, 
despite slightly higher `time_taken`.

---

### **Instance 5**
- **Greedy**:  
  `total_cost = 299.90`, `time_taken = 0.0`, `distance_gap = 126.86`  
- **Branch and Cut**:  
  `total_cost = 284.92`, `time_taken = 0.010`, `distance_gap = 107.14`  
- **Selected Algorithm**: **Branch and Cut**  
  **Reason**: "Branch and Cut" achieves a lower `total_cost` (284.92 vs 299.90) and a better `distance_gap`, 
despite slightly higher `time_taken`.

---

### **Instance 6**
- **Greedy**:  
  `total_cost = 322.40`, `time_taken = 0.0`, `distance_gap = 158.43`  
- **Branch and Cut**:  
  `total_cost = 322.40`, `time_taken = 0.013`, `distance_gap = 158.43`  
- **Selected Algorithm**: **Greedy**  
  **Reason**: Both algorithms have the same `total_cost` and `distance_gap`, but "Greedy" has a lower `time_taken` 
(0.0 vs 0.013).

---

### **Instance 7**
- **Greedy**:  
  `total_cost = 428.49`, `time_taken = 0.0`, `distance_gap = 126.66`  
- **Branch and Cut**:  
  `total_cost = 428.49`, `time_taken = 0.011`, `distance_gap = 126.66`  
- **Selected Algorithm**: **Greedy**  
  **Reason**: Both algorithms have the same `total_cost` and `distance_gap`, but "Greedy" has a lower `time_taken` 
(0.0 vs 0.011).

---

### **Instance 8**
- **Greedy**:  
  `total_cost = 407.53`, `time_taken = 0.0`, `distance_gap = 130.69`  
- **Branch and Cut**:  
  `total_cost = 407.53`, `time_taken = 0.012`, `distance_gap = 130.69`  
- **Selected Algorithm**: **Greedy**  
  **Reason**: Both algorithms have the same `total_cost` and `distance_gap`, but "Greedy" has a lower `time_taken` 
(0.0 vs 0.012).

---

### **Instance 9**
- **Greedy**:  
  `total_cost = 330.09`, `time_taken = 0.0`, `distance_gap = 166.43`  
- **Branch and Cut**:  
  `total_cost = 328.73`, `time_taken = 0.015`, `distance_gap = 165.06`  
- **Selected Algorithm**: **Branch and Cut**  
  **Reason**: "Branch and Cut" achieves a lo

[Step 1: Duration 44.58 seconds| Input tokens: 3,127 | Output tokens: 2,581]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

Execution logs:
This is a debug message
Processing data...
Processing completed in 0.00 seconds
Processed data: 84

Out: None

[Step 10: Duration 6.55 seconds| Input tokens: 7,220 | Output tokens: 6,072]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 11 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 11: Duration 3.59 seconds| Input tokens: 7,320 | Output tokens: 6,529]

KeyboardInterrupt: 